# This noteboook converts .cifs to featurized clusters, containing molecular and atomic information for node embedding.
## This requires:

`.cif` files for the crystal and `.mol2` files for the molecular unit

In [1]:
import pandas as pd
import os 
from ase.io import read,write
import random

from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import Draw
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors3D
from rdkit.Chem import AllChem

import numpy as np 

Ptable = Chem.GetPeriodicTable()

Enegs = {"O": 3.44,
        "C": 2.55,
        "N": 3.04,
        "F": 3.98,
        "Cl": 3.16,
        "H": 2.20,
        "P": 2.19,
        "S": 2.58,
        "I": 2.66,
        "Br": 2.96,
        "Na": 0.93,
        "Se": 2.55,
        "Si": 1.95,
        "B": 2.04,
        "K": 0.82}

def get_descriptors(m):

    # try:
    try: 
        AllChem.EmbedMolecule(m, randomSeed=0xf00d) 
    except: 
        return
    # except ArgumentError:
    #     return np.zeros(24)
    R_xyz = m.GetConformer().GetPositions()

    #Molecular Descriptors
    ecc = rdMolDescriptors.CalcEccentricity(m)
    R_gy = rdMolDescriptors.CalcRadiusOfGyration(m)
    molVol = AllChem.ComputeMolVolume(m) / 100
    polarity = get_dipole(m)
    asphericity = rdMolDescriptors.CalcAsphericity(m)
    nats = m.GetNumAtoms() 
    nHBA = rdMolDescriptors.CalcNumLipinskiHBA(m)
    nHBD = rdMolDescriptors.CalcNumLipinskiHBD(m)
    nrings = rdMolDescriptors.CalcNumRings(m)
    nrots = rdMolDescriptors.CalcNumRotatableBonds(m)
    molMass = rdMolDescriptors.CalcExactMolWt(m) / 100
    planarity = Chem.rdMolDescriptors.CalcPBF(m)
    I1 = Descriptors3D.PMI1(m) / 1000
    I2 = Descriptors3D.PMI2(m) / 1000
    I3 = Descriptors3D.PMI3(m) / 1000
    if not Chem.FindMolChiralCenters(m,includeUnassigned=True):
        isChiral = 0
    else:
        isChiral = 1
    M = np.array([molVol,molMass,nats/10,isChiral,nrings,nHBD,nHBA,nrots,
                  planarity,polarity,asphericity,ecc,R_gy,I1,I2,I3])

    #M = M / np.max(M)

    
    #Atomic Descriptors
    atomic_descriptors = np.zeros((nats,24))
    
    for i, atom in enumerate(m.GetAtoms()):
        sym = atom.GetSymbol()
        num = atom.GetAtomicNum()
        mass = atom.GetMass() / 10
        valence = atom.GetTotalValence()
        ring = int(atom.IsInRing())
        Rvdw = Ptable.GetRvdw(sym)
        isHBD = int(is_hbd(atom))
        isHBA = int(is_hba(atom))

        if sym in Enegs:
            en = Enegs[sym]
        else:
            raise Exception(f"electronegativity for {sym} not found")
        A = np.array([num,mass,isHBD,isHBA,valence,Rvdw,ring,en])
        V = np.concatenate((A,M),axis=0)
        atomic_descriptors[i] = V
        #V = torch.cat((A,M))
    return atomic_descriptors


def is_hba(atom):
    return atom.GetTotalNumHs() == 0 and (atom.GetAtomicNum() == 7 or atom.GetAtomicNum() == 8 or atom.GetAtomicNum() == 16)

def is_hbd(atom):
    return atom.GetTotalNumHs() > 0 and atom.GetAtomicNum() == 7 or atom.GetAtomicNum() == 8

    
def get_dipole(m):
    charges = rdMolDescriptors.CalcEEMcharges(m)
    # Initialize variables for the net molecular dipole
    net_dipole_x = 0.0
    net_dipole_y = 0.0
    net_dipole_z = 0.0
    
    # Calculate bond dipoles and update the net dipole
    for bond in m.GetBonds():
        atom1 = bond.GetBeginAtom()
        atom2 = bond.GetEndAtom()
        #charge1 = atom1.GetDoubleProp("_GasteigerCharge")
        #charge2 = atom2.GetDoubleProp("_GasteigerCharge")
        charge1 = charges[atom1.GetIdx()]
        charge2 = charges[atom2.GetIdx()]
        bond_vector = m.GetConformer().GetAtomPosition(atom2.GetIdx()) - m.GetConformer().GetAtomPosition(atom1.GetIdx())
        
        bond_dipole_x = (charge1 - charge2) * bond_vector.x
        bond_dipole_y = (charge1 - charge2) * bond_vector.y
        bond_dipole_z = (charge1 - charge2) * bond_vector.z
    
        net_dipole_x += bond_dipole_x
        net_dipole_y += bond_dipole_y
        net_dipole_z += bond_dipole_z
    
    # Calculate the magnitude of the net dipole
    net_molecular_dipole = (net_dipole_x**2 + net_dipole_y**2 + net_dipole_z**2)**0.5
    
    #print(f"Net Molecular Dipole Moment: {net_molecular_dipole:.4f} Debye")
    return net_molecular_dipole


# Gather .cif and create molecular clusters for featurization
## cifs are currently pulled directly from the CSD 

[critic2](https://aoterodelaroza.github.io/critic2/)

Handles identification of molecules from crystallographic data, generation of molecular clusters, and more 
molecular crystal xforms

In [6]:
import subprocess
import os
import shutil

clusterloc = os.path.join(os.getcwd(),'cluster')
cifloc = os.path.join(os.getcwd(),'cif')

os.makedirs('cluster',exist_ok=True)

with open('makeclust.cri','w') as f:
    for cif in os.listdir(cifloc):
        ID = cif.replace('.cif','')
        clusterfile = os.path.join(clusterloc,'{}.xyz'.format(ID))
        ciffile = os.path.join(cifloc,cif)
        f.write('CRYSTAL {} \n WRITE {} SPHERE 10 MOLMOTIF\n'.format(ciffile,clusterfile))

devnull = open(os.devnull, 'w')
critic_command = ' '.join(['critic2','makeclust.cri'])
subprocess.run(critic_command,shell=True,stdout=devnull)

CompletedProcess(args='critic2 makeclust.cri', returncode=0)

# Featurize data according to Tuckermann's 2023 paper. Descriptors from the get_descriptors() function. Initially stores all data in a Pandas dataframe, converts to .pkl for building graphs.

In [ ]:
clusterloc = os.path.join(os.getcwd(),'cluster')
molecloc = os.path.join(os.getcwd(),'mol2')
df = pd.DataFrame(columns=['Structure','Descriptors','Coordinates','Label'])
os.makedirs('odd_molecs',exist_ok=True)
#Real Structures
for f in os.listdir(molecloc):
    label = 1
    ID = f.replace('.mol2','')
    clusterfile = os.path.join(clusterloc,ID + '.xyz')
    
    molfile = os.path.join(molecloc,f)
    m = Chem.MolFromMol2File(molfile,removeHs=False)
    V = get_descriptors(m)
    if V is None:
        print(ID, 'Embedding failed, quarantine structure in odd_molecs/')
        os.remove(clusterfile)
        shutil.move(molfile,os.path.join('odd_molecs',ID+'.mol2'))
        continue
    atoms = read(clusterfile)
    coordinates = atoms.get_positions()

    N = int(len(coordinates) / V.shape[0])
    #allV = np.tile(V,(N,1))

    #SOMETIMES THE ATOMIC/MOLECULAR DESCRIPTORS ARE NOT TILED TO ALL COORDINATES!!! FIX THIS!
    allV = np.tile(V,(N,1))

    if len(allV) != len(coordinates):
        print('Tiling failed for {}'.format(ID))
        continue

    row = [ID,allV,coordinates,label]
    df.loc[len(df)]=row

df.to_pickle('GNN_DATA.pkl')

# Misc stuff handling crystal standardization - not actually important here.


In [22]:
#Standardizing cifs

from pymatgen.core.structure import Structure,Molecule
from pyxtal.io import search_molecules_in_crystal
from ase import Atoms

def align_canonical(molecs):
    return

def inertial_angles(m_i):
    """This is for a cartesian basis, would the crystal lattice vectors be better???"""
    
    cart_units = np.eye(3,3)
    a1 = np.arccos(np.dot(cart_units[0], m_i[0]))
    a2 = np.arccos(np.dot(cart_units[1], m_i[1]))
    a3 = np.arccos(np.dot(cart_units[2], m_i[1]))
    return a1*180/np.pi ,a2*180/np.pi ,a3*180/np.pi


def params_to_matrix(lengths,angles):
    a = lengths[0]
    b = lengths[1]
    c = lengths[2]
    alpha = angles[0]
    beta = angles[1]
    gamma = angles[2]
    n = (np.cos(alpha) - np.cos(gamma) * np.cos(beta)) / np.sin(gamma)
    M = np.array([a, 0, 0, 
                  b * np.cos(gamma), b * np.sin(gamma), 0, 
                  c * np.cos(beta), c * n, c * np.sqrt(np.sin(beta) ** 2 - n ** 2)])
    M = M.reshape(3, 3)
    return M
    
struc = Structure.from_file(ciffile)


cluster = Molecule.from_file('cluster/POVFIP.xyz')
cluster_species = [n.as_dict()['name'] for n in cluster.sites]
cluster_coords = [n.as_dict()['xyz'] for n in cluster.sites]

molecs = search_molecules_in_crystal(struc)
coms = [np.linalg.norm(m.center_of_mass) for m in molecs]
canon = molecs[coms.index(min(coms))]

species = [n.as_dict()['name'] for n in canon.sites]
coords = [n.as_dict()['xyz'] for n in canon.sites]

ase_canon = Atoms(species,coords)
com = ase_canon.get_center_of_mass()
abc_com = np.matmul(np.linalg.inv(struc.lattice.matrix).T,com)
m_i = ase_canon.get_moments_of_inertia(vectors=True)[1]

aligned_canon = [np.matmul(m_i,c) for c in coords]
aligned = Atoms(species,aligned_canon)
write('aligned_molec.xyz',aligned)

phi,psi,theta = inertial_angles(m_i)
abc = struc.lattice.abc
angles = struc.lattice.angles

full_cell_parameters = [abc[0],abc[1],abc[2],angles[0],angles[1],angles[2],
                        abc_com[0],abc_com[1],abc_com[2],phi,psi,theta                       
      
                       ]

aligned_canon = [np.matmul(m_i,c) for c in coords]
aligned = Atoms(species,aligned_canon)
write('aligned_molec.xyz',aligned)

FileNotFoundError: [Errno 2] No such file or directory: 'cluster/POVFIP.xyz'

In [6]:
df

,Structure,Descriptors,Coordinates,Label
